In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import matplotlib.pylab as plt

In [2]:
train_data_dir = 'Correctly Named Dataset'
train_data_dir1 = 'ASLAlphabet/asl_alphabet_train/asl_alphabet_train'
val_data_dir1 = 'ASLAlphabet/asl_alphabet_test/asl_alphabet_test'

In [3]:
IMG_SIZE = (224,224)
IMG_SHAPE = IMG_SIZE + (3,)
batch_size = 16

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,   
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    batch_size=batch_size,
    target_size=IMG_SIZE,
    class_mode="sparse",
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    batch_size=batch_size,
    target_size=IMG_SIZE,
    class_mode="sparse",
    subset='validation'
)
labels = (validation_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

Found 1944 images belonging to 26 classes.
Found 486 images belonging to 26 classes.
{0: 'A', 1: 'B', 2: 'C', 3: 'D', 4: 'E', 5: 'F', 6: 'G', 7: 'H', 8: 'I', 9: 'J', 10: 'K', 11: 'L', 12: 'M', 13: 'N', 14: 'O', 15: 'P', 16: 'Q', 17: 'R', 18: 'S', 19: 'T', 20: 'U', 21: 'V', 22: 'W', 23: 'X', 24: 'Y', 25: 'Z'}


In [5]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=IMG_SHAPE))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(26, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 16)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 32)      4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 93312)             0         
                                                                 
 dense (Dense)               (None, 64)               

In [6]:
model.compile(
  optimizer='adam',
  loss='sparse_categorical_crossentropy',
  metrics=['acc'])

callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

# Train the model
epochs = 40
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data = validation_generator,
    callbacks=[callback]
)

model.save('custom.h5')

Epoch 1/40


2023-10-20 16:09:09.265070: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-10-20 16:09:09.808656: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


122/122 [==============================] - ETA: 0s - loss: 3.3287 - acc: 0.0340

2023-10-20 16:09:55.559108: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


122/122 [==============================] - 55s 402ms/step - loss: 3.3287 - acc: 0.0340 - val_loss: 3.2514 - val_acc: 0.0412
Epoch 2/40
 83/122 [===================>..........] - ETA: 12s - loss: 3.2497 - acc: 0.0371

KeyboardInterrupt: 

In [ ]:
# Load and preprocess a single image
img = keras.preprocessing.image.load_img('x.jpg', target_size=(224, 224))
img_array = keras.preprocessing.image.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)


prediction = model.predict(img_array)
print(prediction)